# Adding Baselines to the already existant REINFORCE Algorithm

\begin{array}
A_t &= G_t - V(s_t) \\
\nabla J(\theta) &= \sum_{t=0}^{T-1} \nabla 
\log \pi(a_t/s_t) \times A_t
\end{array}


In [11]:
import gymnasium as gym
import torch
import torch.nn as nn
from torch.autograd import Variable
env = gym.make('CartPole-v1')

# Let's create a custom environment
https://www.youtube.com/watch?v=bD6V3rcr_54 is the reference for this.An RL Environment to adjust the temperature of your shower

In [1]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
import random

In [27]:
# creating placeholder functions
class ShowerEnv(Env):
    def __init__(self):
        # defining the actions
        self.action_space = Discrete(3) # 3 discrete actions - meddling with the shower handle
        # Temperature Array
        self.observation_space = Box(low=np.array([0]),high=np.array([100]))
        # Set start temperature
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self,action):
        self.state += action -1
        self.shower_length -= 1
        
        if self.state >=37 and self.state <=38:
            reward = 1
        else:
            reward = -1
        if self.shower_length<= 0:
            truncated = True
            terminated = True
        else:
            truncated = False
            terminated = False
        self.state += random.randint(-1,1)
        info = {}
        return self.state, reward, terminated, truncated, info
    def render(self):
        None
    def reset(self):
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        dummy='dum'
        info = {}
        return self.state, info

In [79]:
class PolicyNetwork():
    def __init__(self,n_state,n_action,n_hidden=50,lr=0.001):
        '''
        forward prop
        '''
        self.model = nn.Sequential(
        nn.Linear(n_state,n_hidden),
        nn.ReLU(),
        nn.Linear(n_hidden,n_action),
        nn.Softmax(),
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr)
        
    def predict(self,s):
        '''
        Compute the action probabilities of state s using
     the learning model
        @param s: input state
        @return: predicted policy
        '''
        return self.model(torch.tensor([float(s)]))
    
    def update(self, returns, log_probs):
        policy_gradient = []
        for log_prob, Gt in zip(log_probs,returns):
            policy_gradient.append(-log_prob*Gt)
        loss = torch.stack(policy_gradient).sum()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def get_action(self, s):
        '''
        @param s: input state
        @return: the selected action and log probability
        '''
        probs = self.predict(s)
        action = torch.multinomial(probs, 1).item()
        log_prob = torch.log(probs[action])
        return action,log_prob
        
    
def reinforce(env,estimator,n_episode,gamma=1.0):
    """
    REINFORCE algorithm
    @param env: Gym environment
    @param estimator: policy network
    @param n_episode: number of episodes
    @param gamma: the discount factor
    """
    for episode in range(n_episode):
        log_probs = []
        rewards = []
        state, info = env.reset()
        state = torch.tensor([float(state)])
        while True:
            action,log_prob = estimator.get_action(state)
            next_state,reward,terminated,truncated,info = env.step(action)
            total_reward_episode[episode] += reward
            log_probs.append(log_prob)
            rewards.append(reward)
            if terminated or truncated:
                returns = []
                Gt = 0 
                pw = 0 # this is essentially the power
                for reward in rewards[::-1]:
                    Gt += gamma ** pw * reward
                    pw += 1
                    returns.append(Gt)
                returns = returns[::-1]
                returns = torch.tensor(returns)
                returns = (returns - returns.mean()) / (returns.std() + 1e-9) # some sort of standardisation
                estimator.update(returns, log_probs)
                print('Episode: {}, total reward: {}'.format(
                    episode, total_reward_episode[episode]))
                break
            state = next_state

            

In [80]:
env=ShowerEnv()

/home/prathith/miniconda3/envs/torch/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [81]:
torch.tensor(np.array(env.reset()[0]))

tensor(37)

In [82]:
torch.tensor([41])

tensor([41])

In [83]:

n_state = env.observation_space.shape[0]
n_action = env.action_space.n
n_hidden = 128
lr = 0.0007
gamma = 0.95
entropic_factor = 0.001
n_episode = 8000
total_reward_episode = [0]*n_episode

In [84]:
policy_net = PolicyNetwork(n_state, n_action, n_hidden, lr)

In [85]:
reinforce(env,estimator=policy_net,n_episode=n_episode,gamma=gamma)


Episode: 0, total reward: -60
Episode: 1, total reward: -52
Episode: 2, total reward: -58
Episode: 3, total reward: -54
Episode: 4, total reward: -58
Episode: 5, total reward: -50
Episode: 6, total reward: -60
Episode: 7, total reward: -54
Episode: 8, total reward: -56
Episode: 9, total reward: -60
Episode: 10, total reward: -60
Episode: 11, total reward: -60
Episode: 12, total reward: -56
Episode: 13, total reward: -54
Episode: 14, total reward: -56
Episode: 15, total reward: -52
Episode: 16, total reward: -60
Episode: 17, total reward: -58
Episode: 18, total reward: -60
Episode: 19, total reward: -56
Episode: 20, total reward: -52
Episode: 21, total reward: -54
Episode: 22, total reward: -56
Episode: 23, total reward: -56
Episode: 24, total reward: -60
Episode: 25, total reward: -60
Episode: 26, total reward: -56
Episode: 27, total reward: -54
Episode: 28, total reward: -56
Episode: 29, total reward: -58
Episode: 30, total reward: -54
Episode: 31, total reward: -54
Episode: 32, total

Episode: 271, total reward: -60
Episode: 272, total reward: -56
Episode: 273, total reward: -60
Episode: 274, total reward: -60
Episode: 275, total reward: -60
Episode: 276, total reward: -54
Episode: 277, total reward: -60
Episode: 278, total reward: -58
Episode: 279, total reward: -60
Episode: 280, total reward: -56
Episode: 281, total reward: -60
Episode: 282, total reward: -54
Episode: 283, total reward: -54
Episode: 284, total reward: -58
Episode: 285, total reward: -54
Episode: 286, total reward: -58
Episode: 287, total reward: -58
Episode: 288, total reward: -54
Episode: 289, total reward: -60
Episode: 290, total reward: -60
Episode: 291, total reward: -56
Episode: 292, total reward: -60
Episode: 293, total reward: -56
Episode: 294, total reward: -54
Episode: 295, total reward: -60
Episode: 296, total reward: -60
Episode: 297, total reward: -50
Episode: 298, total reward: -52
Episode: 299, total reward: -54
Episode: 300, total reward: -60
Episode: 301, total reward: -52
Episode:

Episode: 545, total reward: -56
Episode: 546, total reward: -60
Episode: 547, total reward: -58
Episode: 548, total reward: -60
Episode: 549, total reward: -58
Episode: 550, total reward: -56
Episode: 551, total reward: -60
Episode: 552, total reward: -60
Episode: 553, total reward: -58
Episode: 554, total reward: -58
Episode: 555, total reward: -56
Episode: 556, total reward: -60
Episode: 557, total reward: -58
Episode: 558, total reward: -60
Episode: 559, total reward: -58
Episode: 560, total reward: -54
Episode: 561, total reward: -60
Episode: 562, total reward: -56
Episode: 563, total reward: -58
Episode: 564, total reward: -60
Episode: 565, total reward: -58
Episode: 566, total reward: -58
Episode: 567, total reward: -60
Episode: 568, total reward: -52
Episode: 569, total reward: -60
Episode: 570, total reward: -58
Episode: 571, total reward: -58
Episode: 572, total reward: -60
Episode: 573, total reward: -58
Episode: 574, total reward: -56
Episode: 575, total reward: -58
Episode:

Episode: 807, total reward: -60
Episode: 808, total reward: -58
Episode: 809, total reward: -56
Episode: 810, total reward: -54
Episode: 811, total reward: -58
Episode: 812, total reward: -54
Episode: 813, total reward: -60
Episode: 814, total reward: -58
Episode: 815, total reward: -56
Episode: 816, total reward: -54
Episode: 817, total reward: -58
Episode: 818, total reward: -60
Episode: 819, total reward: -60
Episode: 820, total reward: -58
Episode: 821, total reward: -60
Episode: 822, total reward: -58
Episode: 823, total reward: -60
Episode: 824, total reward: -56
Episode: 825, total reward: -60
Episode: 826, total reward: -56
Episode: 827, total reward: -58
Episode: 828, total reward: -52
Episode: 829, total reward: -60
Episode: 830, total reward: -56
Episode: 831, total reward: -56
Episode: 832, total reward: -60
Episode: 833, total reward: -60
Episode: 834, total reward: -58
Episode: 835, total reward: -58
Episode: 836, total reward: -58
Episode: 837, total reward: -52
Episode:

Episode: 1070, total reward: -54
Episode: 1071, total reward: -56
Episode: 1072, total reward: -58
Episode: 1073, total reward: -60
Episode: 1074, total reward: -60
Episode: 1075, total reward: -60
Episode: 1076, total reward: -60
Episode: 1077, total reward: -60
Episode: 1078, total reward: -60
Episode: 1079, total reward: -50
Episode: 1080, total reward: -60
Episode: 1081, total reward: -60
Episode: 1082, total reward: -60
Episode: 1083, total reward: -58
Episode: 1084, total reward: -60
Episode: 1085, total reward: -58
Episode: 1086, total reward: -58
Episode: 1087, total reward: -58
Episode: 1088, total reward: -58
Episode: 1089, total reward: -58
Episode: 1090, total reward: -56
Episode: 1091, total reward: -56
Episode: 1092, total reward: -58
Episode: 1093, total reward: -52
Episode: 1094, total reward: -58
Episode: 1095, total reward: -58
Episode: 1096, total reward: -60
Episode: 1097, total reward: -52
Episode: 1098, total reward: -58
Episode: 1099, total reward: -58
Episode: 1

...

Episode: 6769, total reward: -58
Episode: 6770, total reward: -60
Episode: 6771, total reward: -60
Episode: 6772, total reward: -56
Episode: 6773, total reward: -60
Episode: 6774, total reward: -56
Episode: 6775, total reward: -60
Episode: 6776, total reward: -58
Episode: 6777, total reward: -58
Episode: 6778, total reward: -60
Episode: 6779, total reward: -52
Episode: 6780, total reward: -60
Episode: 6781, total reward: -56
Episode: 6782, total reward: -56
Episode: 6783, total reward: -58
Episode: 6784, total reward: -58
Episode: 6785, total reward: -48
Episode: 6786, total reward: -60
Episode: 6787, total reward: -60
Episode: 6788, total reward: -58
Episode: 6789, total reward: -52
Episode: 6790, total reward: -58
Episode: 6791, total reward: -60
Episode: 6792, total reward: -60
Episode: 6793, total reward: -58
Episode: 6794, total reward: -60
Episode: 6795, total reward: -60
Episode: 6796, total reward: -60
Episode: 6797, total reward: -60
Episode: 6798, total reward: -54
Episode: 6

Episode: 7019, total reward: -60
Episode: 7020, total reward: -60
Episode: 7021, total reward: -58
Episode: 7022, total reward: -58
Episode: 7023, total reward: -58
Episode: 7024, total reward: -56
Episode: 7025, total reward: -58
Episode: 7026, total reward: -60
Episode: 7027, total reward: -46
Episode: 7028, total reward: -56
Episode: 7029, total reward: -60
Episode: 7030, total reward: -50
Episode: 7031, total reward: -58
Episode: 7032, total reward: -58
Episode: 7033, total reward: -54
Episode: 7034, total reward: -60
Episode: 7035, total reward: -60
Episode: 7036, total reward: -58
Episode: 7037, total reward: -58
Episode: 7038, total reward: -58
Episode: 7039, total reward: -60
Episode: 7040, total reward: -60
Episode: 7041, total reward: -56
Episode: 7042, total reward: -60
Episode: 7043, total reward: -60
Episode: 7044, total reward: -58
Episode: 7045, total reward: -60
Episode: 7046, total reward: -56
Episode: 7047, total reward: -58
Episode: 7048, total reward: -60
Episode: 7

Episode: 7268, total reward: -60
Episode: 7269, total reward: -56
Episode: 7270, total reward: -58
Episode: 7271, total reward: -56
Episode: 7272, total reward: -60
Episode: 7273, total reward: -56
Episode: 7274, total reward: -56
Episode: 7275, total reward: -60
Episode: 7276, total reward: -54
Episode: 7277, total reward: -60
Episode: 7278, total reward: -60
Episode: 7279, total reward: -60
Episode: 7280, total reward: -54
Episode: 7281, total reward: -58
Episode: 7282, total reward: -60
Episode: 7283, total reward: -60
Episode: 7284, total reward: -54
Episode: 7285, total reward: -58
Episode: 7286, total reward: -58
Episode: 7287, total reward: -60
Episode: 7288, total reward: -58
Episode: 7289, total reward: -60
Episode: 7290, total reward: -58
Episode: 7291, total reward: -60
Episode: 7292, total reward: -60
Episode: 7293, total reward: -52
Episode: 7294, total reward: -58
Episode: 7295, total reward: -58
Episode: 7296, total reward: -60
Episode: 7297, total reward: -54
Episode: 7

Episode: 7529, total reward: -60
Episode: 7530, total reward: -52
Episode: 7531, total reward: -58
Episode: 7532, total reward: -56
Episode: 7533, total reward: -56
Episode: 7534, total reward: -56
Episode: 7535, total reward: -60
Episode: 7536, total reward: -58
Episode: 7537, total reward: -60
Episode: 7538, total reward: -60
Episode: 7539, total reward: -60
Episode: 7540, total reward: -56
Episode: 7541, total reward: -52
Episode: 7542, total reward: -58
Episode: 7543, total reward: -58
Episode: 7544, total reward: -60
Episode: 7545, total reward: -54
Episode: 7546, total reward: -60
Episode: 7547, total reward: -60
Episode: 7548, total reward: -54
Episode: 7549, total reward: -58
Episode: 7550, total reward: -56
Episode: 7551, total reward: -60
Episode: 7552, total reward: -60
Episode: 7553, total reward: -56
Episode: 7554, total reward: -52
Episode: 7555, total reward: -56
Episode: 7556, total reward: -58
Episode: 7557, total reward: -54
Episode: 7558, total reward: -60
Episode: 7

Episode: 7794, total reward: -54
Episode: 7795, total reward: -56
Episode: 7796, total reward: -54
Episode: 7797, total reward: -60
Episode: 7798, total reward: -54
Episode: 7799, total reward: -60
Episode: 7800, total reward: -58
Episode: 7801, total reward: -58
Episode: 7802, total reward: -56
Episode: 7803, total reward: -60
Episode: 7804, total reward: -58
Episode: 7805, total reward: -60
Episode: 7806, total reward: -60
Episode: 7807, total reward: -54
Episode: 7808, total reward: -60
Episode: 7809, total reward: -60
Episode: 7810, total reward: -56
Episode: 7811, total reward: -56
Episode: 7812, total reward: -56
Episode: 7813, total reward: -56
Episode: 7814, total reward: -60
Episode: 7815, total reward: -60
Episode: 7816, total reward: -52
Episode: 7817, total reward: -60
Episode: 7818, total reward: -60
Episode: 7819, total reward: -60
Episode: 7820, total reward: -50
Episode: 7821, total reward: -60
Episode: 7822, total reward: -58
Episode: 7823, total reward: -56
Episode: 7

In [67]:
policy_net.predict(torch.tensor([1.9]))

/tmp/ipykernel_440/3560645607.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return self.model(torch.tensor(s))
/home/prathith/miniconda3/envs/torch/lib/python3.11/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([0.2585, 0.3375, 0.4040], grad_fn=<SoftmaxBackward0>)

In [ ]:
# creating placeholder functions
class ShowerEnvNew(Env):
    def __init__(self):
        # defining the actions
        self.action_space = Discrete(3) # 3 discrete actions - meddling with the shower handle
        # Temperature Array
        self.observation_space = Bo```x(low=np.array([0]),high=np.array([100]))
        # Set start temperature
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self,action):
        self.state += action -1
        self.shower_length -= 1
        
        if self.state >=37 and self.state <=38:
            reward = 1
        else:
            reward = -1
        if self.shower_length<= 0:
            truncated = True
            terminated = True
        else:
            truncated = False
            terminated = False
        self.state += random.randint(-1,1)
        info = {}
        return self.state, reward, terminated, truncated, info
    def render(self):
        None
    def reset(self):
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        info = {}
        return self.state, info 

In [8]:
env = ShowerEnv()

In [13]:
n_episode = 100

for episode in range(n_episode):

    action = env.action_space.sample()
    new_state, reward, terminated, truncated,info = env.step(action)
    print(new_state, reward, terminated, truncated,info )

38 1 True True {}
38 1 True True {}
39 1 True True {}
41 -1 True True {}
40 -1 True True {}
40 -1 True True {}
40 -1 True True {}
41 -1 True True {}
41 -1 True True {}
42 -1 True True {}
43 -1 True True {}
41 -1 True True {}
40 -1 True True {}
41 -1 True True {}
41 -1 True True {}
40 -1 True True {}
40 -1 True True {}
40 -1 True True {}
42 -1 True True {}
40 -1 True True {}
40 -1 True True {}
41 -1 True True {}
42 -1 True True {}
44 -1 True True {}
46 -1 True True {}
47 -1 True True {}
47 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
44 -1 True True {}
45 -1 True True {}
45 -1 True True {}
47 -1 True True {}
46 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
44 -1 True True {}
42 -1 True True {}
43 -1 True True {}
44 -1 True True {}
45 -1 True True {}
47 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
45 -1 True True {}
43 -1 True True {}
43 -1 True True {}
42 -1 True True {}
40 -1 True True {}
40 -1 True True